# Feature Engineering of Weblog data
1. Count of Days of Week where session happened # humans have a more even spread of activity across the week
2. Count of Time of Day ('wee_hours' - 1am-7am , 'morning' - 8am-11am, 'afternoon' - 12pm-3pm, 'evening' - 4pm-7pm, 'night' - 8pm-12am) # more humans are most active between 9am-11am
3. Most frequent site/url
4. Most frequent referral
5. Percentage of each type of method used
6. Percentage of each type of status 
7. Average byte size
8. Duration for the entire session
9. Total number of requests # bots tend to have more requests made
10. Average duration between each request

In [1]:
import pandas as pd
import zipfile
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
zip_path = r'../data/interim/weblog_p1p2.zip'  
extract_folder = r'../data/interim/weblog/'

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

In [3]:
weblog_df = pd.read_csv(extract_folder + 'weblog_p1p2.csv', parse_dates=["datetime"])
weblog_df = weblog_df[weblog_df['Session_ID'] != '-']

In [4]:
def categorize_time(hour):
    if 0 <= hour < 8:
        return "wee_hours"
    elif 8 <= hour < 12:
        return "morning"
    elif 12 <= hour < 16:
        return "afternoon"
    elif 16 <= hour < 19:
        return "evening"
    else:
        return "night"

weblog_df["day_of_week"] = weblog_df["datetime"].dt.day_name()
weblog_df["hour"] = weblog_df["datetime"].dt.hour
weblog_df["time_category"] = weblog_df["hour"].apply(categorize_time)
weblog_df['time_diff'] = weblog_df.groupby('Session_ID')['datetime'].diff().dt.total_seconds()

In [5]:
# Pivot `method` and `status`
method_pivot = pd.crosstab(weblog_df["Session_ID"], weblog_df["method"]).reset_index()
status_pivot = pd.crosstab(weblog_df["Session_ID"], weblog_df["status"]).reset_index()

# Convert counts to percentages for method and status
method_pivot_percent = method_pivot.set_index('Session_ID').div(method_pivot.set_index('Session_ID').sum(axis=1), axis=0) * 100
status_pivot_percent = status_pivot.set_index('Session_ID').div(status_pivot.set_index('Session_ID').sum(axis=1), axis=0) * 100

# Reset index to get Session_ID back as a column
method_pivot_percent = method_pivot_percent.reset_index()
status_pivot_percent = status_pivot_percent.reset_index()

# Add prefix to column name
method_pivot_percent.columns = ['Session_ID'] + [f'method_{col}' for col in method_pivot_percent.columns[1:]] 
status_pivot_percent.columns = ['Session_ID'] + [f'status_{col}' for col in status_pivot_percent.columns[1:]]

# Pivot `day_of_week` and `time_category`
days_pivot = pd.crosstab(weblog_df["Session_ID"], weblog_df["day_of_week"]).reset_index()
days_pivot.columns = ['Session_ID'] + [f'day_{col}' for col in days_pivot.columns[1:]]
time_pivot = pd.crosstab(weblog_df["Session_ID"], weblog_df["time_category"]).reset_index()
time_pivot.columns = ['Session_ID'] + [f'time_{col}' for col in time_pivot.columns[1:]]

# convert days and time columns to binary values instead of counts
# days_pivot.iloc[:, 1:] = (days_pivot.iloc[:, 1:] > 0).astype(int)
# time_pivot.iloc[:, 1:] = (time_pivot.iloc[:, 1:] > 0).astype(int)

test = pd.concat([method_pivot, status_pivot, days_pivot, time_pivot], axis=1)

In [6]:
# Group by Session_ID
session_grouped = weblog_df.groupby('Session_ID').agg(
    # url = ('url', lambda x: x.mode()[0]), # most frequent URL per session
    # referrer = ('referrer', lambda x: x.mode()[0]),  # most frequent referrer per session
    # avg_byte_size = ('byte_size', 'mean'),  # average byte size per session
    session_duration = ('datetime', lambda x: (x.max() - x.min()).total_seconds()), # total duration per session
    request_count = ('Session_ID', 'size'), # total number of request made per session
    # avg_request_duration = ('time_diff', 'mean') # average time interval between each request
).reset_index()

final_df = (
    session_grouped
    # .merge(method_pivot_percent, on="Session_ID", how="left")
    # .merge(status_pivot_percent, on="Session_ID", how="left")
    # .merge(days_pivot, on="Session_ID", how="left")
    .merge(time_pivot, on="Session_ID", how="left")
)

In [7]:
final_df

,Session_ID,session_duration,request_count,time_afternoon,time_evening,time_morning,time_night,time_wee_hours
0,01o7p78e2bnu1814jn5k4uqke4,31.0,64,64,0,0,0,0
1,03jt2p4bdru20sjb9me2gco6j4,882.0,897,0,0,897,0,0
2,0654rvnjhnr0pvsi3qa3e16avo,112.0,82,82,0,0,0,0
3,06ivkemfgn93qhl5j0vu96rnl4,7.0,13,0,0,13,0,0
4,071tbv7fsev5d64kb0f9jieor6,680.0,401,0,401,0,0,0
...,...,...,...,...,...,...,...,...
808,vqdvioip730lq32umqa85ikehl,537.0,335,0,0,0,0,335
809,vqrt3maidth9lr4df2egocd88g,18.0,37,37,0,0,0,0
810,vtcjrbtjq57mnai4banl61pd25,638.0,371,129,242,0,0,0
811,vu3fio88psda005g91fbjona0v,72.0,96,0,0,96,0,0


In [8]:
# save file to interim
final_df.to_csv('../data/interim/web_log_features.csv', index=False)